In [1]:
from bs4 import BeautifulSoup

import urllib.parse
import urllib.request

import re


class RecipeNotFound(Exception):
	pass


class Marmiton(object):

	@staticmethod
	def search(query_dict):
		"""
		Search recipes parsing the returned html data.
		Options:
		'aqt': string of keywords separated by a white space  (query search)
		Optional options :
		'dt': "entree" | "platprincipal" | "accompagnement" | "amusegueule" | "sauce"  (plate type)
		'exp': 1 | 2 | 3  (plate expense 1: cheap, 3: expensive)
		'dif': 1 | 2 | 3 | 4  (recipe difficultie 1: easy, 4: advanced)
		'veg': 0 | 1  (vegetarien only: 1)
		'rct': 0 | 1  (without cook: 1)
		'sort': "markdesc" (rate) | "popularitydesc" (popularity) | "" (empty for relevance)
		"""
		base_url = "http://www.marmiton.org/recettes/recherche.aspx?"
		query_url = urllib.parse.urlencode(query_dict)

		url = base_url + query_url

		html_content = urllib.request.urlopen(url).read()
		soup = BeautifulSoup(html_content, 'html.parser')

		search_data = []

		articles = soup.findAll("div", {"class": "recipe-card"})

		iterarticles = iter(articles)
		for article in iterarticles:
			data = {}
			try:
				data["name"] = article.find("h4", {"class": "recipe-card__title"}).get_text().strip(' \t\n\r')
				data["description"] = article.find("div", {"class": "recipe-card__description"}).get_text().strip(' \t\n\r')
				data["url"] = article.find("a", {"class": "recipe-card-link"})['href']
				data["rate"] = article.find("span", {"class": "recipe-card__rating__value"}).text.strip(' \t\n\r')
				try:
					data["image"] = article.find('img')['src']
				except Exception as e1:
					pass
			except Exception as e2:
				pass
			if data:
				search_data.append(data)

		return search_data

	@staticmethod
	def __clean_text(element):
		return element.text.replace("\n", "").strip()

	@staticmethod
	def get(uri):
		"""
		'url' from 'search' method.
		 ex. "/recettes/recette_wraps-de-poulet-et-sauce-au-curry_337319.aspx"
		"""
		data = {}

		base_url = "http://www.marmiton.org"
		url = base_url + ("" if uri.startswith("/") else "/") + uri; print("on va rechercher: ", url)
		try:
			html_content = urllib.request.urlopen(url).read()
		except urllib.error.HTTPError as e:
			raise RecipeNotFound if e.code == 404 else e

		soup = BeautifulSoup(html_content, 'html.parser')

		main_data = soup.find("div", {"class": "m_content_recette_main"})
		try:
			name = soup.find("h1", {"class", "main-title "}).get_text().strip(' \t\n\r')
		except:
			name = soup.find("h1", {"class", "main-title"}).get_text().strip(' \t\n\r')

		ingredients = [item.text.replace("\n", "").strip() for item in soup.find_all("li", {"class": "recipe-ingredients__list__item"})]

		try:
			tags = list(set([item.text.replace("\n", "").strip() for item in soup.find('ul', {"class": "mrtn-tags-list"}).find_all('li', {"class": "mrtn-tag"})]))
		except:
			tags = []

		recipe_elements = [
			{"name": "author", "query": soup.find('span', {"class": "recipe-author__name"})},
			{"name": "rate","query": soup.find("span", {"class": "recipe-reviews-list__review__head__infos__rating__value"})},
			{"name": "difficulty", "query": soup.find("div", {"class": "recipe-infos__level"})},
			{"name": "budget", "query": soup.find("div", {"class": "recipe-infos__budget"})},
			{"name": "prep_time", "query": soup.find("span", {"class": "recipe-infos__timmings__value"})},
			{"name": "total_time", "query": soup.find("span", {"class": "title-2 recipe-infos__total-time__value"})},
			{"name": "people_quantity", "query": soup.find("span", {"class": "title-2 recipe-infos__quantity__value"})},
			{"name": "author_tip", "query": soup.find("div", {"class": "recipe-chief-tip mrtn-recipe-bloc "}).find("p", {"class": "mrtn-recipe-bloc__content"}) if soup.find("div", {"class": "recipe-chief-tip mrtn-recipe-bloc "}) else "" },
		]
		for recipe_element in recipe_elements:
			try:
				data[recipe_element['name']] = Marmiton.__clean_text(recipe_element['query'])
			except:
				data[recipe_element['name']] = ""

		try:
			cook_time = Marmiton.__clean_text(soup.find("div", {"class": "recipe-infos__timmings__cooking"}).find("span"))
		except:
			cook_time = "0"

		try:
			nb_comments = Marmiton.__clean_text(soup.find("span", {"class": "recipe-infos-users__value mrtn-hide-on-print"})).split(" ")[0]
		except:
			nb_comments = ""

		steps = []
		soup_steps = soup.find_all("li", {"class": "recipe-preparation__list__item"})
		for soup_step in soup_steps:
			steps.append(Marmiton.__clean_text(soup_step))

		image = soup.find("img", {"id": "af-diapo-desktop-0_img"})['src'] if soup.find("img", {"id": "af-diapo-desktop-0_img"}) else ""

		data.update({
			"ingredients": ingredients,
			"steps": steps,
			"name": name,
			"tags": tags,
			"image": image if image else "",
			"nb_comments": nb_comments,
			"cook_time": cook_time
		})

		return data

In [2]:
#from marmiton import Marmiton, RecipeNotFound

# Search :


In [26]:
def rechercher_sur_mmt(mot_clef,recherche_par_motclef=10,compteur = 0):
    total_list = []
    
    query_options = {
      "aqt": mot_clef,  # Query keywords - separated by a white space
    }
    query_result = Marmiton.search(query_options)
    for i in range(recherche_par_motclef):
        compteur += 1
        try:
            recipe = query_result[i]
            main_recipe_url = recipe['url']
            detailed_recipe = Marmiton.get(main_recipe_url)  # Get the details of the first returned recipe (most relevant in our case)
            for ingredient in detailed_recipe['ingredients']:  # List of ingredients
                #print("- %s" % ingredient)
                total_list.append(ingredient)
        except:
            #print(f"No recipe found for '{query_options['aqt']}'")
            #import sys
            #sys.exit(0)
            compteur =- 1
            pass

    return total_list,compteur

rechercher_sur_mmt("boeuf")

on va rechercher:  http://www.marmiton.org/recettes/recette_boeuf-bourguignon-rapide_19218.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_boeuf-bourguignon_18889.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_boeuf-braise-aux-carottes_13296.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_boeuf-bourguignon-debutant_81513.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_yaourt-a-la-grecque-miel-et-noix_392595.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_samoussa-au-boeuf-rapides_171841.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_boeuf-bourguignon-d-automne_24399.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_rognon-de-boeuf_21697.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_roti-de-boeuf-en-croute_12191.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_focaccia-tomate_392594.aspx


(['800g de boeuf pour bourguignon',
  '100g de lardons',
  "50g de beurre ou 3 cuillères à soupe d'huile",
  '2/3l de vin rouge',
  '2oignons',
  "1gousse d'ail",
  '2cuillères à soupe de farine',
  '1bouquet garni',
  '250g de champignon de Paris (en boîte)',
  'Sel',
  'Poivre',
  '600g de bourguignon',
  '4oignons',
  '4carottes',
  '1bouquet garni',
  '1bouteille de vin rouge assez bon',
  '100g de beurre',
  'Sel',
  'Poivre',
  '1morceau de paleron ou de bourguignon lkg',
  '2oignons',
  '1tranche de lard coupée en dés',
  'Matière grasse',
  '1petite boîte de concentré de tomates',
  'Laurier',
  'Thym',
  'Sauge',
  '1tablette de bouillon de boeuf',
  'Poivre',
  'Sel',
  "2gousses d'ail",
  '6carottes',
  '1kg de boeuf pour bourguignon',
  '100g de lardons',
  "50g de beurre ou 3 cuillères à soupe d'huile",
  '1/3l de vin rouge (bourgogne ou côtes du Rhône)',
  "1/4l d'eau",
  '4carottes',
  '2oignons',
  'Ail (3 ou 4 gousses)',
  '1cuillère à soupe de farine',
  '1bouquet gar

In [27]:

print("")


In [35]:
%%time

fo = open("/home/oem/result_top_100.txt", "r")
Lines = fo.readlines() 
final_liste_of_word = []

count = 0
compteur_val = 0
# Strips the newline character 
for idx, line in enumerate(Lines): 
    #to_ignore = [18,26,28,42,45,46,84,98,99]
    if not idx in to_ignore:
    #print("Line{}: {}".format(count, line.strip())) 
        #if not idx < to_ignore[-1]:
        print(idx, line.split('"')[3])
        resultat_recherche, compteur_val = rechercher_sur_mmt(line.split('"')[3],5,compteur_val)
        for phrase in resultat_recherche:
            for mot in phrase.split():
                mot = ''.join(i for i in mot if not i.isdigit()).split()
                final_liste_of_word.append(mot)
            
print("nombre de recettes traitées : ", compteur_val)

0 Blanquette de veau facile
on va rechercher:  http://www.marmiton.org/recettes/recette_confiture-de-menthe-marocaine_348085.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-aux-roses-de-pommes-et-aux-amandes_348048.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_salade-d-endives-au-surimi-fromage-de-chevre-et-tomates_348096.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_rougail-de-saucisses-de-la-reunion_348038.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_blanquette-de-veau-minute_392055.aspx
1 Ramequins fondants au chocolat
on va rechercher:  http://www.marmiton.org/recettes/recette_ramequins-fondants-au-chocolat_15816.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_ramequins-fondants-chocolat-framboise_33117.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_ramequins-au-chocolat-aux-coeurs-fondants_14734.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_fondant

on va rechercher:  http://www.marmiton.org/recettes/recette_mini-genoises-au-nutella_349083.aspx
16 Sauté de veau au chorizo
on va rechercher:  http://www.marmiton.org/recettes/recette_saute-de-veau-au-chorizo_17027.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_saute-de-veau-au-chorizo-facile_26310.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_saute-de-dinde-au-chorizo_70047.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_saute-de-porc-au-chorizo_67661.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_le-saute-de-veau-aux-champignons_528625.aspx
17 Gâteau au chocolat des écoliers 
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-au-chocolat-des-ecoliers_20654.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-aux-petits-ecoliers_57077.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_gouter-de-l-ecolier_27609.aspx
on va rechercher:  http://www.marmiton.org/recette

on va rechercher:  http://www.marmiton.org/recettes/recette_panna-cotta-tricolore-au-nutella_348738.aspx
35 Gâteau au yaourt
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-au-yaourt_12719.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-au-yaourt-simple_26217.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-au-yaourt-et-nutella_48257.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-au-yaourt-facile_73370.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-glace-au-yaourt-par-laurent-mariotte_392672.aspx
36 Chouquettes
on va rechercher:  http://www.marmiton.org/recettes/recette_chouquettes_30622.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_chouquettes-au-cookeo_382439.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_chouquettes-simples_13761.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_chouquettes-au-chocolat_308076.aspx
on 

on va rechercher:  http://www.marmiton.org/recettes/recette_endives-caramelisees-au-jambon_38355.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_risotto-jambon-coquillettes_392596.aspx
54 Tarte frangipane aux poires et au chocolat
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-frangipane-aux-poires-et-au-chocolat_15765.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-aux-poires-sur-frangipane-chocolat-pralin_82581.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-a-la-frangipane-et-aux-poires_19215.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-frangipane-au-chocolat_16682.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-amandine-poires-chocolat_372648.aspx
55 Poulet basquaise
on va rechercher:  http://www.marmiton.org/recettes/recette_poulet-basquaise_16969.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_poulet-basquaise_28694.aspx
on va recherc

on va rechercher:  http://www.marmiton.org/recettes/recette_fondue-aux-poireaux-au-companion_382909.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-aux-oignons-et-fondue-de-poireaux-de-niinii_528443.aspx
70 Crevettes au curry express
on va rechercher:  http://www.marmiton.org/recettes/recette_crevettes-au-curry-express_12645.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_curry-de-crevettes-express_26177.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_curry-de-crevette-aux-poireaux-express_231687.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_poisson-au-curry-express_59121.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_paella-express_528557.aspx
71 Gâteau d'anniversaire
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-d-anniversaire_59610.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_gateau-d-anniversaire_16870.aspx
on va rechercher:  http://www.marmiton.org

86 Muffins à la banane
on va rechercher:  http://www.marmiton.org/recettes/recette_muffins-a-la-banane_14745.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_muffins-a-la-banane-legers_228113.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_muffins-a-la-banane-faciles_18292.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_muffins-a-la-banane-version-light_90116.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_muffins-a-l-orange-et-au-nutella_349102.aspx
87 Tarte aux pommes
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-aux-pommes_18588.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-aux-pommes-a-l-alsacienne_11457.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-aux-pommes-miel-et-amandes_15227.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-aux-pommes-facile_92486.aspx
on va rechercher:  http://www.marmiton.org/recettes/recette_tarte-au

In [39]:
final_liste_of_word_clean = [m[0].lower() for m in final_liste_of_word if len(m)>0]
mots = ['râpée','d\'eau','au','goût','noir','blanc','vos','verre','fraîche','jaune','pincée','de','g','+','à','cl','cuillère','la','en','poudre','sachet','doux','ou','semoule','cuillère','selon']
mots_a_eviter = mots + [x+'s' for x in mots]
final_liste_of_word_clean2 = [x for x in final_liste_of_word_clean if x not in mots_a_eviter]

In [40]:
from collections import Counter
count = Counter(final_liste_of_word_clean2)
count.most_common(150)

[('sucre', 328),
 ('soupe', 261),
 ('beurre', 235),
 ('farine', 220),
 ('sel', 219),
 ('oeufs', 200),
 ('poivre', 147),
 ('café', 131),
 ('lait', 123),
 ('crème', 119),
 ('chocolat', 103),
 ('levure', 92),
 ('pâte', 75),
 ("d'huile", 69),
 ("d'olive", 66),
 ("d'ail", 58),
 ("d'oeuf", 52),
 ('et', 51),
 ('râpé', 47),
 ('vanillé', 44),
 ('liquide', 44),
 ('gousses', 42),
 ('chimique', 42),
 ('citron', 39),
 ('pour', 39),
 ('oignons', 39),
 ('feuilletée', 39),
 ('oeuf', 38),
 ("d'amandes", 38),
 ('moutarde', 38),
 ('pommes', 37),
 ('kg', 35),
 ('ml', 35),
 ('oignon', 34),
 ('(ou', 33),
 ('vanille', 32),
 ('tomates', 32),
 ('tranches', 31),
 ('huile', 31),
 ('gruyère', 29),
 ('miel', 29),
 ('bouillon', 28),
 ('muscade', 28),
 ('boîte', 27),
 ('terre', 27),
 ('vin', 26),
 ('glace', 26),
 ('gousse', 25),
 ('(', 25),
 ('pomme', 25),
 ('le', 23),
 ('boeuf', 23),
 ('tomate', 23),
 ('fromage', 23),
 ('l', 22),
 ('brisée', 22),
 ('fondu', 22),
 ('persil', 22),
 ('noix', 21),
 ('bouquet', 20),
 ('

In [18]:
 ('soupe', 261),
 ('beurre', 235),
 ('oeufs', 200),
 ('lait', 123),
 ('crème', 119),
 ('pâte', 75),
 ("d'ail", 58),
 ('râpé', 47),
 ('citron', 39),
 ('oignons', 39),
 ('feuilletée', 39),
 ('moutarde', 38),
 ('pommes', 37),
 ('oignon', 34),
 ('tomates', 32),
 ('tranches', 31),
 ('gruyère', 29),
 ('terre', 27),
 ('vin', 26),
 ('pomme', 25),
 ('boeuf', 23),
 ('tomate', 23),
 ('fromage', 23),
 ('brisée', 22),
 ('persil', 22),
 ('jambon', 20),
 ('mascarpone', 19),
 ('yaourt', 17),
 ('lardons', 16),
  ('champignon', 14),
 ('échalotes', 14),
 ('saumon', 14),
 ('poivron', 14),
 ('carotte', 13),
('sucre', 328),
 ('soupe', 261),
 ('beurre', 235),
 ('farine', 220),
 ('sel', 219),
 ('oeufs', 200),
 ('poivre', 147),
 ('café', 131),
 ('lait', 123),
 ('crème', 119),
 ('chocolat', 103),
 ('levure', 92),
 ('pâte', 75),
 ("d'huile", 69),
 ("d'olive", 66),
 ("d'ail", 58),
 ("d'oeuf", 52),
 ('et', 51),
 ('râpé', 47),
 ('vanillé', 44),
 ('liquide', 44),
 ('gousses', 42),
 ('chimique', 42),
 ('citron', 39),
 ('pour', 39),
 ('oignons', 39),
 ('feuilletée', 39),
 ('oeuf', 38),
 ("d'amandes", 38),
 ('moutarde', 38),
 ('pommes', 37),
 ('kg', 35),
 ('ml', 35),
 ('oignon', 34),
 ('(ou', 33),
 ('vanille', 32),
 ('tomates', 32),
 ('tranches', 31),
 ('huile', 31),
 ('gruyère', 29),
 ('miel', 29),
 ('bouillon', 28),
 ('muscade', 28),
 ('boîte', 27),
 ('terre', 27),
 ('vin', 26),
 ('glace', 26),
 ('gousse', 25),
 ('(', 25),
 ('pomme', 25),
 ('le', 23),
 ('boeuf', 23),
 ('tomate', 23),
 ('fromage', 23),
 ('l', 22),
 ('brisée', 22),
 ('fondu', 22),
 ('persil', 22),
 ('noix', 21),
 ('bouquet', 20),
 ('dessert', 20),
 ('cannelle', 20),
 ('jambon', 20),
 ('mascarpone', 19),
 ('/cuillère', 19),
 ('du', 18),
 ('les', 17),
 ('fumé', 17),
 ('yaourt', 17),
 ('cacao', 16),
 ('mou', 16),
 ('cassonade', 16),
 ('jus', 16),
 ('feuilles', 16),
 ('thym', 16),
 ('lardons', 16),
 ('haché', 16),
 ('/sachet', 16),
 ('maïzena', 16),
 ('entiers', 16),
 ('pot', 15),
 ('pain', 15),
 ('/l', 15),
 ('frais', 15),
 ('citrons', 15),
 ('champignon', 14),
 ('échalotes', 14),
 ('garni', 14),
 ('(facultatif)', 14),
 ('rouge', 14),
 ('herbes', 14),
 ('saumon', 14),
 ('épaisse', 14),
 ('rouges', 14),
 ('rhum', 14),
 ('poivron', 14),
 ('nestlé', 13),
 ('roux', 13),
 ("d'épices", 13),
 ('carotte', 13),
 ('vert', 13),
 ('morceaux', 13),
 ('sirop', 12),
 ('paquet', 12),
 ('pâtes', 12),
 ('provence', 12),
 ('lard', 12),
 ('fin', 12),
 ('%', 12),
 ('cerises', 12),
 ('non', 11),
 ('viande', 11),
 ('porc', 11),
 ('or', 11),
 ('forte', 11),
 ('bien', 11),
 ('cube', 10),
 ('g)', 10),
 ('framboises', 10),
 ('tablette', 10),
 ("d'extrait", 10),
 ('petite', 10),
 ('filet', 10),
 ('hachée', 10),
 ('laurier', 10),
 ('courgette', 10),
 ('herta®', 10),
 ('gélatine', 10),
 ('entier', 10),
 ('nutella®', 10),
 ('.kg', 10),
 ('moule', 10),
 ("d'oignon", 10),
 ('veau', 9),
 ('paris', 9),
 ('vinaigre', 9),
 ('noisette', 9),
 ('branche', 9),
 ('concentré', 9),
 ('feuilletées', 9),
 ('thon', 9),
 ('curry', 9),
 ('poulet', 9),
 ('confites', 9),
 ('gingembre', 9),
 ('piment', 9),
 ('cru', 9),
 ('canard', 9),

['râpée',
 "d'eau",
 'au',
 'goût',
 'noir',
 'blanc',
 'vos',
 'fraîche',
 'jaune',
 'pincée',
 'de',
 'g',
 '+',
 'à',
 'cl',
 'cuillère',
 'la',
 'en',
 'poudre',
 'sachet',
 'doux',
 'ou',
 'semoule',
 'cuillère',
 'selon',
 'jaune',
 'râpées',
 "d'eaus",
 'aus',
 'goûts',
 'noirs',
 'blancs',
 'voss',
 'fraîches',
 'jaunes',
 'pincées',
 'des',
 'gs',
 '+s',
 'às',
 'cls',
 'cuillères',
 'las',
 'ens',
 'poudres',
 'sachets',
 'douxs',
 'ous',
 'semoules',
 'cuillères',
 'selons',
 'jaunes']

In [136]:
final_liste_of_word

[['g'],
 ['de'],
 ['chocolat'],
 ['noir'],
 ['+'],
 [],
 ['carrés'],
 ['('],
 ['g)'],
 ['oeufs'],
 ['g'],
 ['de'],
 ['sucre'],
 ['semoule'],
 ['g'],
 ['de'],
 ['beurre'],
 ['doux'],
 ['cuillère'],
 ['à'],
 ['soupe'],
 ['de'],
 ['farine'],
 ['g'],
 ['de'],
 ['chocolat'],
 ['oeufs'],
 ['g'],
 ['de'],
 ['sucre'],
 ['g'],
 ['de'],
 ['beurre'],
 ['g'],
 ['de'],
 ['farine'],
 ['g'],
 ['de'],
 ['framboises'],
 ['oeufs'],
 ['g'],
 ['de'],
 ['sucre'],
 ['roux'],
 ['sachet'],
 ['de'],
 ['sucre'],
 ['vanillé'],
 ['g'],
 ['de'],
 ['mascarpone'],
 ['biscuits'],
 ['à'],
 ['la'],
 ['cuillère'],
 ['cl'],
 ['de'],
 ['café'],
 ['noir'],
 ['non'],
 ['sucré'],
 ['g'],
 ['de'],
 ['cacao'],
 ['amer'],
 ['g'],
 ['de'],
 ['mascarpone'],
 ['oeufs'],
 ['cuillères'],
 ['à'],
 ['soupe'],
 ['de'],
 ['sucre'],
 ['semoule'],
 ['g'],
 ['de'],
 ['sucre'],
 ['vanillé'],
 ['Biscuit'],
 ['Pavesinis'],
 ['(italiens)'],
 ['ou'],
 ['langues'],
 ['de'],
 ['chats,'],
 ['à'],
 ['la'],
 ['limite'],
 ['biscuits'],
 ['à'],
 ['la'

In [84]:
[ for n in final_liste_of_word[0]]

[['g', 'de', 'chocolat', 'noir', '+', 'carrés', '(', 'g)'],
 ['oeufs'],
 ['g', 'de', 'sucre', 'semoule'],
 ['g', 'de', 'beurre', 'doux'],
 ['cuillère', 'à', 'soupe', 'de', 'farine'],
 ['g', 'de', 'chocolat'],
 ['oeufs'],
 ['g', 'de', 'sucre'],
 ['g', 'de', 'beurre'],
 ['g', 'de', 'farine'],
 ['g', 'de', 'framboises']]

In [51]:
!ls /home/oem/esult_top_100.txt

ls: impossible d'accéder à '/home/oem/esult_top_100.txt': Aucun fichier ou dossier de ce type
